In [63]:
import numpy as np
import random

class Structure:
    def __init__(self, seed, seed_next_possible, portion, destination):
        self.history = None
        self.data = None
        self.rect = []
        self.next_possibles = None
        
        self.portion = portion
        self.seed = seed
        self.seed_next_possible = seed_next_possible
        
        self.cleanUp()
        self.destination = destination
        self.preprocess()

        #print("initialized data: \n", self.data)
    
    def preprocess(self):
        start = self.seed_next_possible[0]
        direction = start[-1]
        new_vertex_clean = np.array([start[0], start[1], start[2]])
        self.add_vertex(new_vertex_clean)
        
        span = 2 + int(random.random() * 5 % 5)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, direction)
            self.add_vertex(new_vertex_clean)
        
        self.remove_possible(0)
        self.history = np.append(self.history, self.data[1:], axis=0)
        self.rect.append(block_to_rect(self.data, self.portion))
        self.data = np.array([[0,0,0]])
        
        self.to_dest(new_vertex_clean)


    def to_dest(self, new_vertex_clean):
        x_dist = abs(self.destination[0] - new_vertex_clean[0])
        x_start = min(self.destination[0], new_vertex_clean[0])
        startPos = np.array([x_start, new_vertex_clean[1], new_vertex_clean[2]])
        scale = np.array([x_dist, 0.1, 0.1])
        rect1 = rect(startPos, scale)
        
        y_dist = abs(self.destination[1] - new_vertex_clean[1])
        y_start = min(self.destination[1], new_vertex_clean[1])
        startPos2 = np.array([self.destination[0], y_start, new_vertex_clean[2]])
        scale2 = np.array([0.1,y_dist, 0.1])
        rect2 = rect(startPos2, scale2)
        
        self.rect.append(rect1)
        self.rect.append(rect2)
        
        
    def cleanUp(self):
        self.history = self.seed
        self.data = np.array([[0.0,0.0,0.0]])
        self.data = np.append(self.data, self.seed, axis = 0)
        self.data = np.array([[0.0,0.0,0.0]])
        self.next_possibles = self.seed_next_possible

  
    def cost_func(self, pos):
        return random.random()
  
    def add_vertex(self, next_vertex):
        x = next_vertex[0]
        y = next_vertex[1]
        z = next_vertex[2]

        self.data = np.vstack([self.data, np.array([x,y,z])])

    def remove_possible(self, next_index):
        self.next_possibles = np.delete(self.next_possibles, next_index, axis=0)
  
  #define direction:
  #x=-1 -> 0, x=1 -> 1, y=-1 -> 2, y=1 -> 3, z=-1 -> 4, z=1 -> 5,
    def get_next_possible(self, available_contact, direction):
        result = np.array([0,0,0,0])

        available_contact = np.append(available_contact, 0)
        if direction == 0 or direction == 1:
            result = np.vstack([result, available_contact + np.array([0.0,0.1,0.0,3])])
            result = np.vstack([result, available_contact + np.array([0.0,-0.1,0.0,2])])
            result = np.delete(result, 0, axis =0)
            return result

        if direction == 2 or direction == 3:
            result = np.vstack([result, available_contact + np.array([0.1,0.0,0.0,1])])
            result = np.vstack([result, available_contact + np.array([-0.1,0.0,0.0,1])])
            result = np.delete(result, 0, axis =0)
            return result

    
    def get_last_direction(self,direction):
        count =direction[1]*1 + direction[2]*2
        return count
  
    def get_vertex_forward(self, new_vertex_clean, direction):
        if direction == 0:
            return new_vertex_clean + np.array([-0.1,0.0,0.0])

        if direction == 1:
            return new_vertex_clean + np.array([0.1,0.0,0.0])

        if direction == 2:
            return new_vertex_clean + np.array([0.0,-0.1,0.0])

        if direction == 3:
            return new_vertex_clean + np.array([0.0,0.1,0.0])
    
    def generate(self, steps):
        for i in range(steps):
            #print("step ", i+1)
            self.process()
            self.history = np.append(self.history, self.data[1:], axis=0)
            self.rect.append(block_to_rect(self.data, self.portion))
            self.data = np.array([[0,0,0]])



    def process(self):
        #step 1: for all available contacts, assume they will make a turn  
        #print("next_possibles", self.next_possibles)

        #step2: create a 1d array that stores cost for each possible point
        costs = np.array([])
        for next_possible in self.next_possibles:
            costs = np.append(costs, self.cost_func(next_possible))

        #print("costs", costs)

        #step3 find the next_vertex with smallest cost
        next_index = np.argpartition(costs, -1)[-1:]
        next_vertex = self.next_possibles[next_index] 

        #print("next_vertex", next_vertex)

        #step4 add next_vertex to data
        new_direction = next_vertex[0][3]
        #print("new_direction", new_direction)
        new_vertex_clean = np.array([next_vertex[0][0], next_vertex[0][1], next_vertex[0][2]])
        #print("new_vertex_clean", new_vertex_clean)
        self.add_vertex(new_vertex_clean)
        
        span = 2 + int(random.random() * 5 % 5)
        for i in range(span):
            new_vertex_clean = self.get_vertex_forward(new_vertex_clean, new_direction)
            self.add_vertex(new_vertex_clean)

        #print("data: \n", self.data)

        #step5 update next_possible
        self.remove_possible(next_index)
        #print("next_possibles, removed old", self.next_possibles)
        #print("next_vertex", new_vertex_clean)
        #print("new_direction", new_direction)
        new_next_possibles = self.get_next_possible(new_vertex_clean, new_direction)
        self.next_possibles = np.append(self.next_possibles,new_next_possibles, axis=0)
        #print("next_possibles, add new", self.next_possibles)


In [64]:
seed = np.array([[0.0,0.0,.00], [0.0,0.1,0.0],[0.0,0.2,0.0],[0.0,0.3,0.0], [0.0,0.4,0.0]])
seed_next_possible = np.array([[-0.1,0.4,0.0,0],[0.1,0.4,0.0,1]])


In [65]:
sa = Structure(seed, seed_next_possible, 0.5, np.array([0.4,0.9]))

sa.generate(5)

In [66]:
sa.history

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e-01,  0.00000000e+00],
       [ 0.00000000e+00,  2.00000000e-01,  0.00000000e+00],
       [ 0.00000000e+00,  3.00000000e-01,  0.00000000e+00],
       [ 0.00000000e+00,  4.00000000e-01,  0.00000000e+00],
       [-1.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [-2.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [-3.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [ 1.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [ 2.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [ 3.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [ 3.00000000e-01,  3.00000000e-01,  0.00000000e+00],
       [ 3.00000000e-01,  2.00000000e-01,  0.00000000e+00],
       [ 3.00000000e-01,  1.00000000e-01,  0.00000000e+00],
       [ 3.00000000e-01,  2.77555756e-17,  0.00000000e+00],
       [ 3.00000000e-01, -1.00000000e-01,  0.00000000e+00],
       [ 3.00000000e-01, -2.00000000e-01

In [51]:
seed2 = np.array([[0.0,0.0,0.0], [0.1,0.0,0.0],[0.2,0.0,0.0],[0.3,0.0,0.0], [0.4,0.0,0.0]])
seed2_next_possible = np.array([[0.4,0.1,0.0,3],[0.4,-0.1,0.0,2]])
sa2 = Structure(seed2, seed2_next_possible, 0.5)

sa2.generate(5)

In [52]:
def clean_history(data):
    result = None
    for i in data:
        x = round(i[0],1)
        y = round(i[1],1)
        z = round(i[2],1)
        result = 

In [54]:
sa.history = np.round(sa.history, 1)

In [55]:
sa.history

array([[ 0. ,  0. ,  0. ],
       [ 0. ,  0.1,  0. ],
       [ 0. ,  0.2,  0. ],
       [ 0. ,  0.3,  0. ],
       [ 0. ,  0.4,  0. ],
       [-0.1,  0.4,  0. ],
       [-0.2,  0.4,  0. ],
       [-0.3,  0.4,  0. ],
       [-0.4,  0.4,  0. ],
       [-0.5,  0.4,  0. ],
       [-0.6,  0.4,  0. ],
       [-0.7,  0.4,  0. ],
       [-0.7,  0.3,  0. ],
       [-0.7,  0.2,  0. ],
       [-0.7,  0.1,  0. ],
       [-0.7,  0. ,  0. ],
       [-0.8,  0. ,  0. ],
       [-0.7,  0. ,  0. ],
       [-0.6,  0. ,  0. ],
       [-0.6, -0.1,  0. ],
       [-0.6, -0.2,  0. ],
       [-0.6, -0.3,  0. ],
       [-0.6, -0.4,  0. ],
       [-0.6, -0.5,  0. ],
       [-0.6, -0.6,  0. ],
       [-0.6, -0.7,  0. ],
       [-0.7, -0.7,  0. ],
       [-0.6, -0.7,  0. ],
       [-0.5, -0.7,  0. ],
       [-0.4, -0.7,  0. ],
       [-0.3, -0.7,  0. ],
       [-0.2, -0.7,  0. ]])

In [43]:
sa2.history

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e-01,  0.00000000e+00,  0.00000000e+00],
       [ 2.00000000e-01,  0.00000000e+00,  0.00000000e+00],
       [ 3.00000000e-01,  0.00000000e+00,  0.00000000e+00],
       [ 4.00000000e-01,  0.00000000e+00,  0.00000000e+00],
       [ 4.00000000e-01,  1.00000000e-01,  0.00000000e+00],
       [ 4.00000000e-01,  2.00000000e-01,  0.00000000e+00],
       [ 4.00000000e-01,  3.00000000e-01,  0.00000000e+00],
       [ 5.00000000e-01,  3.00000000e-01,  0.00000000e+00],
       [ 6.00000000e-01,  3.00000000e-01,  0.00000000e+00],
       [ 7.00000000e-01,  3.00000000e-01,  0.00000000e+00],
       [ 8.00000000e-01,  3.00000000e-01,  0.00000000e+00],
       [ 8.00000000e-01,  4.00000000e-01,  0.00000000e+00],
       [ 8.00000000e-01,  5.00000000e-01,  0.00000000e+00],
       [ 8.00000000e-01,  6.00000000e-01,  0.00000000e+00],
       [ 8.00000000e-01,  7.00000000e-01,  0.00000000e+00],
       [ 8.00000000e-01,  2.00000000e-01

In [67]:
for i in sa.rect:
    i.info()

start_x: -0.3 start_y: 0.4 start_z: 0.0
scale_x: 0.2 scale_y: 0.05 scale_z: 0.05
 
start_x: -0.3 start_y: 0.4 start_z: 0.0
scale_x: 0.7 scale_y: 0.1 scale_z: 0.1
 
start_x: 0.4 start_y: 0.4 start_z: 0.0
scale_x: 0.1 scale_y: 0.5 scale_z: 0.1
 
start_x: 0.1 start_y: 0.4 start_z: 0.0
scale_x: 0.2 scale_y: 0.05 scale_z: 0.05
 
start_x: 0.3 start_y: -0.3 start_z: 0.0
scale_x: 0.05 scale_y: 0.6 scale_z: 0.05
 
start_x: 0.2 start_y: -0.3 start_z: 0.0
scale_x: 0.3 scale_y: 0.05 scale_z: 0.05
 
start_x: 0.4 start_y: -0.3 start_z: 0.0
scale_x: 0.6 scale_y: 0.05 scale_z: 0.05
 
start_x: 0.5 start_y: -1.0 start_z: 0.0
scale_x: 0.05 scale_y: 0.6 scale_z: 0.05
 


In [45]:
def parallel_score(history_a, history_b):
    score = 0
    parallel_pts = []
    
    for i in history_a:
        for j in history_b:
            if(i[0] == j[0] and i[1] == j[1]):
                parallel_pts.append((i[0],i[1]))
                score += 5
            elif(i[0] == j[0] or i[1] == j[1]):
                score += 1
    return score, parallel_pts

In [46]:
(score, parallel_pts)= parallel_score(sa2.history, sa.history)
score

41

In [10]:
len(parallel_pts)

4

In [97]:
def block_to_rect(buffer, portion):
    start = buffer[1]
    end = buffer[-1]
    x_start = min(start[0],end[0])
    y_start = min(start[1],end[1])    
    z_start = min(start[2],end[2])
    
    x_scale = max(abs(start[0] - end[0]), 0.1 * portion)
    y_scale = max(abs(start[1] - end[1]), 0.1 * portion)
    z_scale = max(abs(start[2] - end[2]), 0.1 * portion)
    
    if(x_scale > 0.1 * portion):
        x_scale += 0.1 * portion
        
    if(y_scale > 0.1 * portion):
        y_scale += 0.1 * portion
        
    if(z_scale > 0.1 * portion):
        z_scale += 0.1 * portion

    startPos = np.array([x_start,y_start,z_start])
    scale = np.array([x_scale,y_scale,z_scale])
    
    tempt = rect(startPos, scale)
    return tempt


In [95]:
class rect:
    def __init__(self, startPos, scale): 
        self.start_x = round(startPos[0],2);
        self.start_y = round(startPos[1],2);
        self.start_z = round(startPos[2],2);
        
        self.scale_x = round(scale[0],2);
        self.scale_y = round(scale[1],2);
        self.scale_z = round(scale[2],2);
    
    def info(self):
        print("start_x:",self.start_x,"start_y:",self.start_y,"start_z:",self.start_z)
        print("scale_x:",self.scale_x,"scale_y:",self.scale_y,"scale_z:",self.scale_z)
        print(" ")
    

In [96]:
b1 = np.array([[0,0,0], [0,1,0],[0,2,0],[0,3,0], [0,4,0]])
b2 = np.array([[0,0,1], [0,1,1],[0,2,1],[0,3,1], [0,4,1]])

In [79]:
c = np.append(b1, b2, axis=0)
c

array([[0, 0, 0],
       [0, 1, 0],
       [0, 2, 0],
       [0, 3, 0],
       [0, 4, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 2, 1],
       [0, 3, 1],
       [0, 4, 1]])

In [64]:
rect_list = []

In [65]:
rect_list.append(block_to_rect(b1))
rect_list.append(block_to_rect(b2))
rect_list

[<__main__.rect at 0x7fbf60789ac0>, <__main__.rect at 0x7fbf802414f0>]

In [37]:
pos = np.array([2.0,2.0,4.0])
np.array([[0.0,0.0,0.0]])

In [38]:
pos[0]

2.0

In [45]:
def gen_seed(pos):
    seed = pos
    for i in range(1,5):
        tempt = np.array([pos[0]+0.1*i,pos[1],pos[2]])
        seed = np.vstack([seed, tempt])
    return seed

In [46]:
a=gen_seed(pos)

In [56]:
a[-1][0]

2.4

In [51]:
def get_next_possible(pos):
    tempt1 = np.array([pos[0],pos[1]+0.1,pos[2],4])
    tempt2 = np.array([pos[0],pos[1]-0.1,pos[2],3])
    
    return np.vstack([tempt1, tempt2])

In [52]:
pos_b = a[-1]

In [53]:
x = get_next_possible(pos_b)
x

array([[2.4, 2.1, 4. , 4. ],
       [2.4, 1.9, 4. , 3. ]])